In [1]:
import sys
from Load import *
from UE import *
from Satellite import *
from Oracle import *
import time


def initial_assignment(UEss, Satss, oracle):
    C = UEss[0].coverage_info
    if oracle is None:
        # random assign
        for ueid in UEss:
            ue = UEss[ueid]
            possible_satellites = np.where(C[ueid, :, 0] == 1)[0]
            assert (len(possible_satellites) > 0)
            satid = random.choice(possible_satellites)
            ue.serving_satellite = Satss[satid]
            ue.serving_satellite_history.append(satid)
    else:
        for ueid in UEss:
            ue = UEss[ueid]
            satid = oracle.query_init_satellite(ueid)
            assert (C[ueid, satid, 0] == 1)
            ue.serving_satellite = Satss[satid]
            ue.serving_satellite_history.append(satid)


def monitor_timestamp(env):
    while True:
        if env.now % 100 == 0:
            print(f"Current simulation time {env.now}", file=sys.stderr)
        yield env.timeout(1)


if __name__ == "__main__":
    # configuration
    # When oracle_simulation is False, the oracle_assignment could be True/False
    # When oracle_simulation is True, the oracle_assignment must be True
    max_access_opportunity = 1
    max_access_slots = 100
    oracle_assignment = False
    oracle_simulation = False
    if oracle_simulation:
        assert (oracle_assignment)
    random.seed(10)

    # Loading scenarios
    beginning = time.time()
    UEs_template, satellites_template, C = load_scenario()
    DURATION = C.shape[2]
    env = simpy.Environment()

    assignment_oracle = None
    simulation_oracle = None
    if oracle_assignment:
        assignment_oracle = Oracle()
    if oracle_simulation:
        simulation_oracle = assignment_oracle

    UEs = {}
    for ue_template in UEs_template:
        ID = ue_template.ID
        if ID not in UEs:
            UEs[ID] = UE(
                identity=ID,
                position_x=ue_template.x,
                position_y=ue_template.y,
                coverage_info=C,
                oracle=simulation_oracle,
                env=env
            )
        else:
            print("ERROR")

    Satellites = {}
    for sat_template in satellites_template:
        ID = sat_template.ID
        if ID not in Satellites:
            Satellites[ID] = Satellite(
                identity=ID,
                position_x=sat_template.x,
                position_y=sat_template.y,
                height=sat_template.h,
                coverage_r=sat_template.r,
                velocity=sat_template.v,
                sind=sat_template.sind,
                cosd=sat_template.cosd,
                coverage_info=C,
                max_access_opportunity = max_access_opportunity,
                max_access_slots = max_access_slots,
                env=env
            )
        else:
            print("ERROR")

    for ueid in UEs:
        UEs[ueid].satellites = Satellites

    for satid in Satellites:
        Satellites[satid].UEs = UEs
        Satellites[satid].satellites = Satellites

    print(f"Loading scenario in the simulation takes {time.time() - beginning}")
    # ========= UE, SAT objectives are ready =========
    initial_assignment(UEs, Satellites, assignment_oracle)

    env.process(monitor_timestamp(env))

    print('============= Experiment Log =============')
    env.run(until=DURATION)
    print('============= Experiment Ends =============')
    counter = allCounters(Satellites, UEs)

Loading scenario in the simulation takes 0.0674898624420166
============= Experiment Log =============
UE 37 sends Satellite 1 the message {"task": "MEASUREMENT_REPORT", "candidates": [2, 3, 4], "from": 37, "to": 1} at 199
Satellite 1 sends Satellite 2 the message {"task": "HANDOVER_REQUEST", "ueid": 37, "from": 1, "to": 2} at 199
Satellite 1 sends Satellite 3 the message {"task": "HANDOVER_REQUEST", "ueid": 37, "from": 1, "to": 3} at 199
Satellite 1 sends Satellite 4 the message {"task": "HANDOVER_REQUEST", "ueid": 37, "from": 1, "to": 4} at 199
Satellite 2 sends Satellite 1 the message {"task": "HANDOVER_RESPONSE", "ueid": 37, "condition": {"access_delay": 84, "ueid": 37, "source": 1, "satid": 2}, "from": 2, "to": 1} at 199
Satellite 3 sends Satellite 1 the message {"task": "HANDOVER_RESPONSE", "ueid": 37, "condition": {"access_delay": 75, "ueid": 37, "source": 1, "satid": 3}, "from": 3, "to": 1} at 199
Satellite 4 sends Satellite 1 the message {"task": "HANDOVER_RESPONSE", "ueid": 3

Current simulation time 0
Current simulation time 100
Current simulation time 200
Current simulation time 300
Current simulation time 400
Current simulation time 500
Current simulation time 600
Current simulation time 700
Current simulation time 800
Current simulation time 900
Current simulation time 1000
Current simulation time 1100
Current simulation time 1200
Current simulation time 1300
Current simulation time 1400
Current simulation time 1500
Current simulation time 1600
Current simulation time 1700
Current simulation time 1800


UE 43 sends Satellite 2 the message {"task": "RANDOM_ACCESS", "from": 43, "to": 2} at 1727
Satellite 2 sends UE 43 the message {"task": "RANDOM_ACCESS_RESPONSE", "from": 2, "to": 43} at 1727
Satellite 2 sends Satellite 0 the message {"task": "HANDOVER_SUCCESS", "ueid": 43, "from": 2, "to": 0} at 1727
Satellite 0 sends Satellite 2 the message {"task": "SN_STATUS_TRANSFER", "ueid": 43, "from": 0, "to": 2} at 1727
Satellite 0 sends Satellite 1 the message {"task": "HANDOVER_CANCEL", "ueid": 43, "from": 0, "to": 1} at 1727
UE 3 sends Satellite 2 the message {"task": "MEASUREMENT_REPORT", "candidates": [3, 4], "from": 3, "to": 2} at 1747
Satellite 2 sends Satellite 3 the message {"task": "HANDOVER_REQUEST", "ueid": 3, "from": 2, "to": 3} at 1747
Satellite 2 sends Satellite 4 the message {"task": "HANDOVER_REQUEST", "ueid": 3, "from": 2, "to": 4} at 1747
Satellite 3 sends Satellite 2 the message {"task": "HANDOVER_RESPONSE", "ueid": 3, "condition": {"access_delay": 54, "ueid": 3, "source": 2

Current simulation time 1900
Current simulation time 2000
Current simulation time 2100
Current simulation time 2200
Current simulation time 2300
Current simulation time 2400
Current simulation time 2500
Current simulation time 2600
Current simulation time 2700
Current simulation time 2800
Current simulation time 2900
Current simulation time 3000
Current simulation time 3100
Current simulation time 3200
Current simulation time 3300
Current simulation time 3400
Current simulation time 3500
Current simulation time 3600
Current simulation time 3700
Current simulation time 3800
Current simulation time 3900
Current simulation time 4000
Current simulation time 4100
Current simulation time 4200
Current simulation time 4300
Current simulation time 4400
Current simulation time 4500


UE 29 sends Satellite 0 the message {"task": "MEASUREMENT_REPORT", "candidates": [1, 2], "from": 29, "to": 0} at 3481
Satellite 0 sends Satellite 1 the message {"task": "HANDOVER_REQUEST", "ueid": 29, "from": 0, "to": 1} at 3481
Satellite 0 sends Satellite 2 the message {"task": "HANDOVER_REQUEST", "ueid": 29, "from": 0, "to": 2} at 3481
Satellite 1 sends Satellite 0 the message {"task": "HANDOVER_RESPONSE", "ueid": 29, "condition": {"access_delay": 81, "ueid": 29, "source": 0, "satid": 1}, "from": 1, "to": 0} at 3481
Satellite 2 sends Satellite 0 the message {"task": "HANDOVER_RESPONSE", "ueid": 29, "condition": {"access_delay": 96, "ueid": 29, "source": 0, "satid": 2}, "from": 2, "to": 0} at 3481
Satellite 0 sends UE 29 the message {"task": "RRC_RECONFIGURATION", "conditions": [{"access_delay": 81, "ueid": 29, "source": 0, "satid": 1}, {"access_delay": 96, "ueid": 29, "source": 0, "satid": 2}], "from": 0, "to": 29} at 3481
UE 29 sends Satellite 0 the message {"task": "RRC_RECONFIGURA

...

UE 23 sends Satellite 8 the message {"task": "MEASUREMENT_REPORT", "candidates": [9, 10], "from": 23, "to": 8} at 15195
Satellite 8 sends Satellite 9 the message {"task": "HANDOVER_REQUEST", "ueid": 23, "from": 8, "to": 9} at 15195
Satellite 8 sends Satellite 10 the message {"task": "HANDOVER_REQUEST", "ueid": 23, "from": 8, "to": 10} at 15195
Satellite 9 sends Satellite 8 the message {"task": "HANDOVER_RESPONSE", "ueid": 23, "condition": {"access_delay": 31, "ueid": 23, "source": 8, "satid": 9}, "from": 9, "to": 8} at 15195
Satellite 10 sends Satellite 8 the message {"task": "HANDOVER_RESPONSE", "ueid": 23, "condition": {"access_delay": 58, "ueid": 23, "source": 8, "satid": 10}, "from": 10, "to": 8} at 15195
Satellite 8 sends UE 23 the message {"task": "RRC_RECONFIGURATION", "conditions": [{"access_delay": 31, "ueid": 23, "source": 8, "satid": 9}, {"access_delay": 58, "ueid": 23, "source": 8, "satid": 10}], "from": 8, "to": 23} at 15195
UE 23 sends Satellite 8 the message {"task": "RR

Current simulation time 16600
Current simulation time 16700
Current simulation time 16800
Current simulation time 16900
Current simulation time 17000
Current simulation time 17100
Current simulation time 17200
Current simulation time 17300
Current simulation time 17400
Current simulation time 17500
Current simulation time 17600
Current simulation time 17700
Current simulation time 17800
Current simulation time 17900
Current simulation time 18000
Current simulation time 18100
Current simulation time 18200


UE 13 sends Satellite 12 the message {"task": "RANDOM_ACCESS", "from": 13, "to": 12} at 16795
Satellite 12 sends UE 13 the message {"task": "RANDOM_ACCESS_RESPONSE", "from": 12, "to": 13} at 16795
Satellite 12 sends Satellite 9 the message {"task": "HANDOVER_SUCCESS", "ueid": 13, "from": 12, "to": 9} at 16795
Satellite 9 sends Satellite 12 the message {"task": "SN_STATUS_TRANSFER", "ueid": 13, "from": 9, "to": 12} at 16795
Satellite 9 sends Satellite 10 the message {"task": "HANDOVER_CANCEL", "ueid": 13, "from": 9, "to": 10} at 16795
Satellite 9 sends Satellite 11 the message {"task": "HANDOVER_CANCEL", "ueid": 13, "from": 9, "to": 11} at 16795
UE 37 sends Satellite 11 the message {"task": "MEASUREMENT_REPORT", "candidates": [12, 13, 14], "from": 37, "to": 11} at 16848
Satellite 11 sends Satellite 12 the message {"task": "HANDOVER_REQUEST", "ueid": 37, "from": 11, "to": 12} at 16848
Satellite 11 sends Satellite 13 the message {"task": "HANDOVER_REQUEST", "ueid": 37, "from": 11, "to": 1

Current simulation time 18300
Current simulation time 18400
Current simulation time 18500
Current simulation time 18600
Current simulation time 18700
Current simulation time 18800
Current simulation time 18900
Current simulation time 19000
Current simulation time 19100
Current simulation time 19200
Current simulation time 19300
Current simulation time 19400
Current simulation time 19500
Current simulation time 19600
Current simulation time 19700
Current simulation time 19800
Current simulation time 19900


UE 25 sends Satellite 10 the message {"task": "MEASUREMENT_REPORT", "candidates": [11, 12], "from": 25, "to": 10} at 19003
Satellite 10 sends Satellite 11 the message {"task": "HANDOVER_REQUEST", "ueid": 25, "from": 10, "to": 11} at 19003
Satellite 10 sends Satellite 12 the message {"task": "HANDOVER_REQUEST", "ueid": 25, "from": 10, "to": 12} at 19003
Satellite 11 sends Satellite 10 the message {"task": "HANDOVER_RESPONSE", "ueid": 25, "condition": {"access_delay": 44, "ueid": 25, "source": 10, "satid": 11}, "from": 11, "to": 10} at 19003
Satellite 12 sends Satellite 10 the message {"task": "HANDOVER_RESPONSE", "ueid": 25, "condition": {"access_delay": 16, "ueid": 25, "source": 10, "satid": 12}, "from": 12, "to": 10} at 19003
Satellite 10 sends UE 25 the message {"task": "RRC_RECONFIGURATION", "conditions": [{"access_delay": 44, "ueid": 25, "source": 10, "satid": 11}, {"access_delay": 16, "ueid": 25, "source": 10, "satid": 12}], "from": 10, "to": 25} at 19003
UE 25 sends Satellite 10 

In [2]:
counter.give_result(100)